In [31]:
import requests
from bs4 import BeautifulSoup
from bs4 import Comment
from PIL import Image
import re
import os


#num 수정

def extact_tistory_blog(url):

    #url = 'https://mvmv3754.tistory.com/3'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')


    titles = soup.select_one('#content > div.inner > div.post-cover > div > h1').text ### 포스팅 글 제목
    special_char = '\/:*?"<>|.'
    for c in special_char:
        if c in titles:
            titles = titles.replace(c, '')  # 특수 문자 제거

    date = soup.select_one('#content > div.inner > div.post-cover > div > span.meta > span.date') ### 등록일

    #if not titles or not date :
    #    continue

    #print(titles.text)    
    #print(date.text)


    ### 포스팅 내용
    entry_content = soup.find('div', {'class':'entry-content'})
    #print(entry_content.get_text())

    res = requests.get(url)
    soup_img = BeautifulSoup(res.content, 'lxml')
    imgs = soup_img.select('img[src^=https]')  # https 로 시작하는 src, '//'로 시작하는 src 제외시킴
    #print(f'이미지 수 : {len(imgs)}')
    #print(imgs)

    # 저장 디렉토리 만들기
    if not os.path.exists('tistoryBackup'):
        os.mkdir('tistoryBackup')
    if not os.path.exists('tistoryBackup/'+str(titles)):
        os.makedirs('tistoryBackup/'+str(titles))

    cnt = 1
    for img in imgs:
        img_url = img['src']

        ## pillow.Image로 이미지 format 알아내기
        imageObj = Image.open(requests.get(img_url, stream=True).raw)
        img_format = imageObj.format
        imge_size = imageObj.size
        #print(f'img_url: {img_url}')
        #print(f'img_format: {img_format}')
        #print(f'imge_size: {imge_size}')
        #print(f'os.path.basename(img_url): {os.path.basename(img_url)}')

        res_img = requests.get(img_url).content
        #print(f'len(이미지): {len(res_img)}')  # requests의 .content는 bytes 타입을 리턴함

        if img_url.split('.')[-1] in ['png', 'jpg']:
            img_name = str(titles) + '_' + str(cnt) + '_' + os.path.basename(img_url)
        else:
            img_name = str(titles) + '_' + str(cnt) + '_' + 'no_filename_img.' + img_format

        #print(img_name)

        if len(res_img) > 100:  # 이미지 용량이 00 bytes 이상인 것만
            with open('./tistoryBackup/' + str(titles) + '/' + img_name, 'wb') as f:
                f.write(res_img)
            cnt += 1

    title_content = str(titles) + '\n' + date.text +  '\n' + entry_content.get_text()
    filename = str(titles) + '_tistory_title_content.txt'
    with open('./tistoryBackup/' + str(titles) + '/' + filename, 'w', encoding='utf-8') as f:
        f.write(title_content)

    print(str(titles) + "의 이미지, 텍스트 백업이 완료되었습니다.")


In [32]:
url ='https://mvmv3754.tistory.com/4'
extact_tistory_blog(url)

망고포도(샤인머스캣)효능과 비싼이유의 이미지, 텍스트 백업이 완료되었습니다.
